In [1]:
import numpy as np
import regex as re
from collections import deque

In [2]:
with open("inp.txt") as f:
    inp = f.read().splitlines()
mmap = inp[:-2]
instructions = inp[-1]

In [3]:
instructions = re.findall('\d+|[A-Z]',instructions)
# instructions

In [4]:
max_x = max((len(x) for x in mmap))
mmap = np.array([list(x.ljust(max_x)) for x in mmap])
mmap = np.vectorize(lambda x: ord(x))(mmap)
mmap[mmap == 32] = -1
mmap[mmap == 46] = 0
mmap[mmap == 35] = 1
# mmap

In [5]:
face_score = {
    ">": 0,
    "v": 1,
    "<": 2,
    "^": 3
}
facings = deque((">","v","<","^"))

In [6]:
loc = 0,np.where(mmap[0] == 0)[0][0]
facing = ">"

In [7]:
def finish(y,x):
    print(f"finished at: row {x}. column {y}. facing {facing}")
    print(
        f"Password: {1000 * x} + {4 * y} + {face_score[facing]}")

In [8]:
def check_new(y,x):
    global mmap

    if mmap[y,x] == 1:
        return 1
    elif mmap[y,x] == 0:
        return 0

    elif mmap[y,x] == -1:
        return -1


In [9]:
for instr in instructions:
    try:
        instr = int(instr)
    except ValueError:
        pass
    if isinstance(instr, int):
        # print(facing, end=" ")
        while instr > 0:
            y,x  = loc
            if x+1 == mmap.shape[1]:
                x = -1
            if y+1 == mmap.shape[0]:
                y = -1
            # print(loc)
            # print(instr, end=" ")
            if facing == ">":
                result = check_new(y, x+1)
                if result == -1:
                    # First 0 in row
                    next_loc = y, np.where(mmap[y] != -1)[0][0]
                    if mmap[next_loc] == 1:
                        instr = 0
                    else:
                        loc = next_loc
                elif result == 1:
                    instr = 0
                else: 
                    loc = y, x+1
            elif facing == "v":
                result = check_new(y+1, x)
                if result == -1:
                    # First 0 in column
                    next_loc = np.where(mmap[:,x] != -1)[0][0], x
                    if mmap[next_loc] == 1:
                        instr = 0
                    else:
                        loc = next_loc
                elif result == 1:
                    instr = 0
                else: 
                    loc = y+1, x
            elif facing == "<":
                result = check_new(y, x-1)
                if result == -1:
                    # Last 0 in row
                    next_loc = y, np.where(mmap[y] != -1)[0][-1]
                    if mmap[next_loc] == 1:
                        instr = 0
                    else:
                        loc = next_loc
                elif result == 1:
                    instr = 0
                else: 
                    loc = y, x-1
            elif facing == "^":
                result = check_new(y-1, x)
                if result == -1:
                    # Last 0 in column
                    next_loc = np.where(mmap[:,x] != -1)[0][-1], x
                    if mmap[next_loc] == 1:
                        instr = 0
                    else:
                        loc = next_loc
                elif result == 1:
                    instr = 0
                else: 
                    loc = y-1, x
            instr -= 1
            # print(loc)

    else:
        # print("\n",facing, end=" ")
        # print(instr, end=" ")
        if instr == "R":
            facings.rotate(-1)
        else:
            facings.rotate(1)
        facing = facings[0]
        # print(facing)

y,x  = loc
print(f"Finished at: Row {y+1}. Column {x+1}. Facing {facing}.")
print(f"Password: {1000 * (y+1) + 4 * (x+1) + face_score[facing]}")


Finished at: Row 27. Column 123. Facing >.
Password: 27492


# Part 2

In [10]:
with open("inp.txt") as f:
    inp = f.read().splitlines()
mmap = inp[:-2]
instructions = inp[-1]

In [11]:
instructions = [int(x) if x.isnumeric() else x for x in re.findall('\d+|[A-Z]', instructions)]

In [12]:
max_x = max((len(x) for x in mmap))
mmap = np.array([list(x.ljust(max_x)) for x in mmap])

In [13]:
class Face:
    def __init__(self, y, x, sides):
        self.offsets = x, y
        self.map = mmap[y:y+50,x:x+50]
        self.sides = sides

    def shift(self, dir, x, y):
        this = self.sides[dir]
        n_face = this[0]
        n_x, n_y = this[1](x,y)
        n_dir = this[2]

        return n_face, n_x, n_y, n_dir

In [14]:
A = 0, 50
B = 0, 100
C = 50, 50
D = 100, 50
E = 100, 0
F = 150, 0

In [15]:
# direction: [new face, new (x,y), new direction]
A_sides = {
    ">": ["B", lambda x,y: (0, y), ">"],
    "v": ["C", lambda x,y: (x, 0), "v"],
    "<": ["E", lambda x,y: (0, 49 - y), ">"],
    "^": ["F", lambda x,y: (0, x), ">"],
}

B_sides = {
    ">": ["D", lambda x,y: (49, 49 - y), "<"],
    "v": ["C", lambda x,y: (49, x), "<"],
    "<": ["A", lambda x,y: (49, y), "<"],
    "^": ["F", lambda x,y: (x, 49), "^"],
}

C_sides = {
    ">": ["B", lambda x,y: (y, 49), "^"],
    "v": ["D", lambda x,y: (x, 0), "v"],
    "<": ["E", lambda x,y: (y, 0), "v"],
    "^": ["A", lambda x,y: (x, 49), "^"]
}

D_sides = {
    ">": ["B", lambda x,y: (49, 49 - y), "<"],
    "v": ["F", lambda x,y: (49, x), "<"],
    "<": ["E", lambda x,y: (49, y), "<"],
    "^": ["C", lambda x,y: (x, 49), "^"],
}

E_sides = {
    ">": ["D", lambda x,y: (0, y), ">"],
    "v": ["F", lambda x,y: (x, 0), "v"],
    "<": ["A", lambda x,y: (0, 49 - y), ">"],
    "^": ["C", lambda x,y: (0, x), ">"],
}

F_sides = {
    ">": ["D", lambda x,y: (y, 49), "^"],
    "v": ["B", lambda x,y: (x, 0), "v"],
    "<": ["A", lambda x,y: (y, 0), "v"],
    "^": ["E", lambda x,y: (x, 49), "^"],
}


In [16]:
faces = {
    "A": Face(*A, A_sides),
    "B": Face(*B, B_sides),
    "C": Face(*C, C_sides),
    "D": Face(*D, D_sides),
    "E": Face(*E, E_sides),
    "F": Face(*F, F_sides),
}

In [17]:
dir_score = {
    ">": 0,
    "v": 1,
    "<": 2,
    "^": 3
}
dirs = deque((">","v","<","^"))

In [18]:
loc = ("A", (0, 0))
dir = ">"

for instr in instructions:
    if isinstance(instr, int):
        while instr > 0:
            # print(instr)
            face, (y, x) = loc
            # print(loc, dir)
            if dir == ">":
                if x+1 == 50:
                    n_face, n_x, n_y, n_dir = faces[face].shift(dir, x, y)
                    if faces[n_face].map[n_y,n_x] == "#":
                        # print(f"Hit wall when moving from {face} to {n_face}!")
                        break
                    loc = (n_face, (n_y, n_x))
                    dir = n_dir
                else:
                    if faces[face].map[y,x+1] == "#":
                        # print("Hit wall")
                        break
                    loc = (face, (y, x+1))
                faces[loc[0]].map[loc[1]] = dir

            elif dir == "v":
                if y+1 == 50:
                    n_face, n_x, n_y, n_dir = faces[face].shift(dir, x, y)
                    if faces[n_face].map[n_y,n_x] == "#":
                        # print(f"Hit wall when moving from {face} to {n_face}!")
                        break
                    loc = (n_face, (n_y, n_x))
                    dir = n_dir
                else:
                    if faces[face].map[y+1,x] == "#":
                        # print("Hit wall")
                        break
                    loc = (face, (y+1, x))
                faces[loc[0]].map[loc[1]] = dir

            elif dir == "<":
                if x-1 == -1:
                    n_face, n_x, n_y, n_dir = faces[face].shift(dir, x, y)
                    if faces[n_face].map[n_y,n_x] == "#":
                        # print(f"Hit wall when moving from {face} to {n_face}!")
                        break
                    loc = (n_face, (n_y, n_x))
                    dir = n_dir
                else:
                    if faces[face].map[y,x-1] == "#":
                        # print("Hit wall")
                        break
                    loc = (face, (y, x-1))
                faces[loc[0]].map[loc[1]] = dir

            elif dir == "^":
                if y-1 == -1:
                    n_face, n_x, n_y, n_dir = faces[face].shift(dir, x, y)
                    if faces[n_face].map[n_y,n_x] == "#":
                        # print(f"Hit wall when moving from {face} to {n_face}!")
                        break
                    loc = (n_face, (n_y, n_x))
                    dir = n_dir
                else:
                    if faces[face].map[y-1,x] == "#":
                        # print("Hit wall")
                        break
                    loc = (face, (y-1, x))
                faces[loc[0]].map[loc[1]] = dir
                    
            instr -= 1


    else:
        # print(instr)
        while not dirs[0] == dir:
            dirs.rotate(1)
        if instr == "R":
            dirs.rotate(-1)
        else:
            dirs.rotate(1)
        dir = dirs[0]

face, (y, x) = loc
print(f"Finished at: Face {face}. Row {y+1}. Column {x+1}. Direction {dir}.")
x_offset, y_offset = faces[face].offsets
print(f"Password: {(1000 * (y+y_offset+1)) + (4 * (x+x_offset+1)) + dir_score[dir]}")

Finished at: Face C. Row 28. Column 22. Direction ^.
Password: 78291
